In [1]:
import os
import codecs

import numpy as np

import matplotlib
import matplotlib.pyplot as plt, mpld3 
# The mpld3 project brings together Matplotlib, the popular Python-based graphing 
#library, and D3js, the popular JavaScript library for creating interactive data visualizations for the web.
mpld3.enable_notebook()

import spacy
# precisa instalar e baixar antes: 
# para windows procurar hunspell em: https://sourceforge.net/projects/ezwinports/files/
# e depois colocar o executável no PATH do windows
# e só depois pip install hunspell
import hunspell 
import pandas as pd

import seaborn as sns
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

# Dados relativos ao artigo: https://sites.icmc.usp.br/taspardo/PROPOR2018-MonteiroEtAl.pdf

In [2]:
def corpus_arquivos(data_dir):
    
    lst_files = []
    for dirpath, dirnames, filenames in os.walk(data_dir):
        for f in filenames:
            if f.endswith(".txt"):
                lst_files.append(os.path.join(dirpath, f))
                
    return lst_files

In [3]:
# pegando os arquivos fakes
lst_fake = corpus_arquivos("Fake.Br Corpus/full_texts/fake/")

# pegando os arquivos verdadeiros
lst_verdadeiros = corpus_arquivos("Fake.Br Corpus/full_texts/true/")

In [4]:
# leitura dos arquivos 
# classes 1: fake / 0: true
# encoding: utf-8 (universal) e iso8859-1 (português)
target = []
file_names = []
text_news = []
for news_file in lst_fake:
    file_names.append(news_file)
    text_news.append(codecs.open(news_file, "r",encoding="iso8859-1").read())
    target.append(1)
    
#text_news = text_news[:100]
#target = target[:100]
#file_names = file_names[:100]

for news_file in lst_verdadeiros:
    file_names.append(news_file)
    text_news.append(codecs.open(news_file, "r", encoding="iso8859-1").read())
    target.append(0)
    
#text_news = text_news[:200]
#target = target[:200]
#file_names = file_names[:200]

print(len(text_news))

7200


# Pré-processamento de cada texto

In [ ]:
import nltk
# podemos remover '' e `` tambem, eles apareceram como muito frequentes em 
# uma primeira execucao então voltei e acrecentei eles aqui
stopwords = nltk.corpus.stopwords.words('portuguese') + ["''","``"]

In [ ]:
# certifique-se que você já baixou o modelo com o comando: 
# python -m spacy download pt_core_news_md
def load_docs():
    model = spacy.load("pt_core_news_md")

    # primeiro passo: tokenizar por sentença cada texto
    model_news = []
    for idx,text in enumerate(text_news):
        #print(idx)
        model_news.append(model(text))
    return model_news

In [ ]:
%time model_news = load_docs()

In [ ]:
print(len(model_news))

# Extração das features

In [ ]:
# As primeira features a serem removidas são relacionadas ao 
# tamanho da sentença

# a partir daí também podemos também pré-processar o texto
tok_news = []
# vamos armazenar as nossas features como dicionario e podemos transformar facilmente em 
# um dataframe
sent_features = {'sent5':[],'sent5_10':[],'sent10':[]}

for doc in model_news:
    # esses serao os contadores para cada doc
    sent5 = 0 
    sent5_10 = 0
    sent10 = 0
    
    tok_doc = []
    for sent in doc.sents:
        
        # desconsiderando stopwords e palavras de tamanho 1
        tok_lst = [tok for tok in sent if len(tok.text) > 1 and tok.text not in stopwords]

        if len(tok_lst) <= 5:
            sent5 += 1
        elif len(tok_lst) > 5 and len(tok_lst) <= 10:
            sent5_10 += 1
        else:
            sent10 += 1
        tok_doc += tok_lst
        
    sent_features['sent5'].append(sent5)
    sent_features['sent5_10'].append(sent5_10) 
    sent_features['sent10'].append(sent10)
    
    tok_news.append(tok_doc)   

In [ ]:
print(tok_news[0]) # dando uma olhda nos tokens do primeiro documento 

In [ ]:
print(sent_features) # vamos ver como ficaram nossas features de sentenças

In [ ]:
# com um histograma podemos ver melhor
ax = plt.hist(sent_features['sent5'], 10)
plt.title('Sentenças menores que 5')

In [ ]:
# com um histograma podemos ver melhor
ax = plt.hist(sent_features['sent5_10'], 10)
plt.title('Sentenças menores que 10 e maior que 5')

In [ ]:
# com um histograma podemos ver melhor
ax = plt.hist(sent_features['sent10'], 10)
plt.title('Sentenças maiores que 10')

# Analisando os termos e extraindo outras features

In [ ]:
# vamos ver os tokens frequentes
# tem algum que quero incluir em stop words?
freq_term = {}
for doc in model_news:
    for tok in doc:
        if tok.text in freq_term:
            freq_term[tok.text] += 1
        else:
            freq_term[tok.text] = 1
            
freq_term_lst = list(freq_term.items())
freq_term_lst.sort(key=lambda tup: tup[1])

In [ ]:
print(freq_term_lst[:10]) # veja que são termos com baixa frequencia, podemos remove-los depois

In [ ]:
print(freq_term_lst[-10:]) # veja como esta a acentuação..isso tem a ver com o encoding) 
# vejam que so eliminei os tokens de tamanho das sentenças, eles continuam no modelo

In [ ]:
# vamos usar o spacy para extrair as entidades nomeadas e nossas features #loc/#palavras, #per/#palavras e 
# #org/#palavras


In [ ]:
ner_features = {'num_per':[],'num_loc':[],'num_org':[]}

for doc in model_news:
    
    num_palavras = len(doc)
    num_per = 0
    num_loc = 0
    num_org = 0
    for ent in doc.ents:
        if ent.label_ == 'PER':
            num_per += 1
        if ent.label_ == 'LOC':
            num_loc += 1
        if ent.label_ == 'ORG':
            num_org += 1
                
     
    ner_features['num_per'].append(num_per / num_palavras)
    ner_features['num_loc'].append(num_loc / num_palavras)
    ner_features['num_org'].append(num_org / num_palavras)
    
print(ner_features)

In [ ]:
# vamos agora para as features de pos tagging
pos_features = {'VERB':[],'ADJ':[],'NOUN':[],'ADV':[]}

for doc in model_news:
    
    verb = 0
    adj = 0
    noun = 0
    adv = 0
    num_palavras = len(doc)
    
    for tok in doc:
        if tok.pos_ == 'VERB':
            verb += 1
        if tok.pos_ == 'ADJ':
            adj += 1
        if tok.pos_ == 'NOUN':
            noun += 1
        if tok.pos_ == 'ADV':
            adv += 1

    pos_features['VERB'].append(verb / num_palavras)
    pos_features['ADJ'].append(adj / num_palavras)
    pos_features['NOUN'].append(noun / num_palavras)
    pos_features['ADV'].append(adv / num_palavras)
    
print(pos_features)

In [ ]:
# vamos contar os termos que estao no nosso dicionario racista

# vamos carregar o dicionario
dicionario_racista = open('dicRacista.txt','r').read().replace('\n','').split(',')
print(dicionario_racista[:5])

In [ ]:
racista_features = {'racista':[]}
for doc in model_news:
    count = 0
    for tok in doc:
        if tok.text in dicionario_racista:
            count += 1
    racista_features['racista'].append(count)
print(racista_features)

In [ ]:
# agora vamos extrair a quantidade de lexicos enviesados
dicionario_vies = {'argumentativo':[],'pressuposicao':[],'possibilidade_necessidade':[],'opiniao_valoracao':[]}

fd_dicionario_vies = open("bias_words.txt","r")
for line in fd_dicionario_vies:
    entry = line.replace("\n","").split(",")
    term = entry[0].strip()
    type_term = entry[1].strip()
    dicionario_vies[type_term].append(term)
    
print(dicionario_vies)

In [ ]:
# depois da lista de termos podemos buscá-los em nossos textos
def extrai_vies_features():
    vies_features = {'argumentativo':[],'pressuposicao':[],'possibilidade_necessidade':[],'opiniao_valoracao':[]}

    for idx, doc in enumerate(model_news):
        # print(idx)
        for type_term in dicionario_vies:
            count = 0
            for term in dicionario_vies[type_term]:

                for sent in doc.sents:
                    if term in sent.text.lower():
                        count += 1
                    
            vies_features[type_term].append(count / len(doc))
        
    return vies_features

In [ ]:
%time vies_features = extrai_vies_features() # demora um minuto e meio para 200 documentos

In [ ]:
# vamos agora analisar os erros ortograficos atraves de dicionario
# embora existam tecnicas sofisticadas de detecção de erros ortográficos, 
# vamos usar o método de dicionário com o hunspell para simplificação da nossa tarefa
hobj = hunspell.HunSpell('pt_BR.dic','pt_BR.aff')

In [ ]:
# so para testar o hunpsell
print(hobj.spell("casa")) # <-- certo: True
print(hobj.spell("caza")) # <-- errado: False

In [ ]:
ort_features = {'erros_ort':[]}
len_features = {'char_len':[]}
for doc in model_news:
    count = 0
    len_count = 0
    for tok in doc:
        len_count += len(tok.text)
        if not(hobj.spell(tok.text)):
            count += 1
    ort_features['erros_ort'].append(count / len(doc))
    len_features['char_len'].append(len_count)
print(ort_features)

In [ ]:
# agora contar os lexicos positivos e negativos
dicionario_pos_neg = {'positivos':[],'negativos':[]}
fd_sent_lexicon = open('oplexicon_v3.0/lexico_v3.0.txt','r')

for line in fd_sent_lexicon:
    entry = line.replace('\n','').split(',')
    # ignorando emoticon e hashtags
    if entry[1] != 'emot' and entry[1] != 'htag':
        if entry[2].strip() == '-1':
            dicionario_pos_neg['negativos'].append(entry[0])
        if entry[2].strip() == '1':
            dicionario_pos_neg['positivos'].append(entry[0])
print(dicionario_pos_neg)

In [ ]:
def extrai_pos_neg():
    pos_neg_features = {'positivos':[],'negativos':[]}

    for doc in model_news:
        positivos = 0
        negativos = 0
        for tok in doc:
            if tok.text in dicionario_pos_neg['positivos']:
                positivos += 1
            if tok.text in dicionario_pos_neg['negativos']:
                negativos += 1
            
        pos_neg_features['positivos'].append(positivos/len(doc))
        pos_neg_features['negativos'].append(negativos/len(doc))
    return pos_neg_features

In [ ]:
%time pos_neg_features = extrai_pos_neg()

In [ ]:
# por fim, coletar os lexicos do emotaix
# estou coletando apenas de 'Super category' (terceira coluna)
dicionario_emotaix = {'ÓDIO':[],'AGRESSIVIDADE':[],'AFEIÇÃO':[],'GENTILEZA':[]}
fd_emotaix = open('Emotaix-pt .csv','r')
header = fd_emotaix.readline()

for line in fd_emotaix:
    entry = line.replace('\n','').split(",")
    if entry[2] in dicionario_emotaix:
        dicionario_emotaix[entry[2]].append(entry[0])
print(dicionario_emotaix)

In [ ]:
emotaix_features = {'ÓDIO':[],'AGRESSIVIDADE':[],'AFEIÇÃO':[],'GENTILEZA':[]}

for doc in model_news:
    odio = 0
    agressividade = 0
    afeicao = 0
    gentileza = 0
    for tok in doc:
        if tok.text in dicionario_emotaix['ÓDIO']:
            odio += 1
        if tok.text in dicionario_emotaix['AGRESSIVIDADE']:
            agressividade += 1
        if tok.text in dicionario_emotaix['AFEIÇÃO']:
            afeicao += 1
        if tok.text in dicionario_emotaix['GENTILEZA']:
            gentileza += 1
            
    emotaix_features['ÓDIO'].append(odio / len(doc))
    emotaix_features['AGRESSIVIDADE'].append(agressividade / len(doc))
    emotaix_features['AFEIÇÃO'].append(afeicao / len(doc))
    emotaix_features['GENTILEZA'].append(gentileza / len(doc))
print(emotaix_features)

# Agora juntar todas as features

In [ ]:
# sent_features
# ner_features 
# pos_features
# racista_features
# vies_features/subjetividade
# ort_features 
# len_features
# pos_neg_features
# emotaix_features
data = {}
data_lst = [sent_features, ner_features, pos_features, 
            racista_features, vies_features, ort_features, 
            len_features, pos_neg_features, emotaix_features]

for ftr in data_lst:
    for k in ftr:
        data[k] = ftr[k]
        
print(data)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [5]:
# agora vamos escrever em arquivo para guardar a extração 
# e não precisar rodar todo este script de novo
#df.to_csv('fake_news_pos.csv',index=False)
df = pd.read_csv("fake_news_pos_full.csv")

# Visualizando as features através de histogramas

In [ ]:
# o searbon tem cinco temas: darkgrid, whitegrid, dark, white, e ticks
sns.set_style("white")
ax = df.hist(bins=30, figsize=(15, 15))

# Visualizar com PCA e T-sne

In [ ]:
# https://cmdlinetips.com/2018/03/pca-example-in-python-with-scikit-learn/


from sklearn import decomposition

In [ ]:
pca = decomposition.PCA(n_components=2)

In [ ]:
pc = pca.fit_transform(df)

In [ ]:
pc_df = pd.DataFrame(data = pc , 
        columns = ['PC1', 'PC2'])
pc_df['Cluster'] = target
pc_df.head()

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
df_explain = pd.DataFrame({'var':pca.explained_variance_ratio_,
             'PC':['PC1','PC2']})
sns.barplot(x='PC',y="var", 
           data=df_explain, color="c");

In [ ]:
# para evitar apagar plots anteriores, podemos usar o subblot
# Retorna o objeto da figura e também o eixo (axes) onde sera plotado
# fica mais facil de manipular o nosso grafico tbm
fig, ax = plt.subplots()

# coordenadas x
# coordenadas y
# c: sequencia de cores, no caso associamos a sequência de cores a nossa classe (fake ou não)
# alpha: O valor de opacidade das cores, entre 0 (transparente) e 1 (totalmente opaco).
# cmap -> color map: mapa de cores. Opções: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
scatter = ax.scatter(pc_df.PC1, pc_df.PC2,
                     c=pc_df.Cluster, 
                     alpha=0.6,
                     cmap=plt.get_cmap("PiYG"))
plt.savefig('pca_fakenews_200.png')

# opção caso queira colocar uma "grade" no seu gráfico    
ax.grid(color='white', linestyle='solid')

N = len(pc_df)
labels = ["Linha %d" % d for d in range(N)]

fig = plt.gcf()
tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels)

mpld3.plugins.connect(fig, tooltip)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)

In [ ]:
df_tsne = tsne.fit_transform(df)

In [ ]:
ax = plt.subplot()

colors_tsne = np.array(target)
# duas classes apenas
num_classes = len(np.unique(target))
# pegando emprestado do seaborn uma paleta de cores
palette = np.array(sns.color_palette("hls", num_classes))

sc = ax.scatter(df_tsne[:,0], df_tsne[:,1], lw=0, s=40, c=palette[colors_tsne.astype(np.int)])
plt.savefig('tsne_fakenews_200.png')

fig = plt.gcf()
tooltip = mpld3.plugins.PointLabelTooltip(sc, labels)

mpld3.plugins.connect(fig, tooltip)

In [ ]:
print(file_names[110])

# Classificando os nossos dados

In [6]:
# Só com o tamanho char_len -> baseline
# vamos usar a validação cruzada com 5 folds, assim como no artigo
from sklearn.model_selection import cross_val_score, cross_validate, ShuffleSplit
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from sklearn.utils import shuffle
df['target'] = target

df = df.sample(frac=1,random_state=0)

df.head()
X = df.drop('target', 1)
y = df.target

In [12]:
print(X.shape, df.shape)

(7200, 23) (7200, 24)


In [13]:
clf = svm.SVC() # vamos usar os parâmetros que são default

In [14]:
scores_svc = cross_validate(clf, X, y, cv=5, scoring=['accuracy','f1','precision','recall'])

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/svm/

In [15]:
# vamos imprimir os nosso scores para cada fold
for m in scores_svc:
    print(m, np.mean(scores_svc[m]))

fit_time 1.6036953926086426
score_time 0.6173890590667724
test_accuracy 0.9134722222222222
train_accuracy 0.9836458333333333
test_f1 0.9089024753624404
train_f1 0.9837167613208683
test_precision 0.9596080835141436
train_precision 0.9794867656420653
test_recall 0.8633333333333333
train_recall 0.9879861111111111


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.6/site-packages/sklea

In [16]:
# agora vamos testar com o classificador que o artigo usou
# que tbm usou com os parâmetros padroes
clf2 = svm.LinearSVC()
scores_linearsvc = cross_validate(clf2, X, y, cv=5, scoring=['accuracy','f1','precision','recall'])

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
 

In [17]:
# vamos olhar nossos resultados
# vamos imprimir os nosso scores para cada fold

# aqui o recall aumentou e a precisao caiu, mas o nosso f1 aumentou um pouco
for m in scores_linearsvc:
    print(m, np.mean(scores_linearsvc[m]))

fit_time 0.583608865737915
score_time 0.008779764175415039
test_accuracy 0.7038888888888889
train_accuracy 0.703923611111111
test_f1 0.7895836766142019
train_f1 0.7893784014792662
test_precision 0.6849077169835934
train_precision 0.6834492625009013
test_recall 0.9838888888888888
train_recall 0.9847222222222222


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.6/site-packages/sklea

In [18]:
# Sera que um metodo ensemble pode se dar melhor?
clf3 = RandomForestClassifier(n_estimators=3)
scores_forest = cross_validate(clf3, X, y, cv=5, scoring=['accuracy','f1','precision','recall'])


In [19]:
# vamos olhar nossos resultados
# vamos imprimir os nosso scores para cada fold

# olha só que legal, alcançamos resultados ainda melhores que o do artigo com um ensemble
# contudo, lembrando que temos apenas 200 documentos!
for m in scores_forest:
    print(m, np.mean(scores_forest[m]))

fit_time 0.03225717544555664
score_time 0.01051921844482422
test_accuracy 0.9863888888888889
train_accuracy 0.9972569444444443
test_f1 0.9864244734945393
train_f1 0.9972616300404009
test_precision 0.9842796751133926
train_precision 0.9958485281773198
test_recall 0.9886111111111111
train_recall 0.9986805555555553


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.6/site-packages/sklea

# Agora vamos analisar as features

In [ ]:
# quais features podem explicar o sucesso da nosso classificador de Random Forest?
# vamos usar o shap para isso
# https://github.com/slundberg/shap
import shap
import sklearn
# load JS visualization code to notebook
shap.initjs()

In [ ]:
# para usar o shap, vamos aqui usar o dado todo para buscar as nossas explicações
# assim precisamos treinar o classificador clf3 no nosso dado

X = df.drop('target', 1)
y = df.target
print(y)

In [ ]:
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
clf4 = RandomForestClassifier(n_estimators=3)
clf4.fit(X_train, y_train)

In [ ]:
# Apenas um experimento da professora com Curva de precisão-recall
y_pred = clf4.predict_proba(X_valid)
pos_probs = y_pred[:, 1]

precision, recall, _ = precision_recall_curve(y_valid, pos_probs)
auc_score = auc(recall, precision)
print('No Skill PR AUC: %.3f' % auc_score)

In [ ]:
explainer = shap.TreeExplainer(clf4)

In [ ]:
# Calculate Shap values
shap_values = explainer.shap_values(X_train)
#explainer.shap_values

In [ ]:
# summarize the effects of all the features
fig = plt.gcf()
shap.summary_plot(shap_values, X_train, feature_names=df.columns,plot_size=(15,7))
fig.savefig('shap_randomforest.png')

# Outra forma de calcular importância de feature com RandomForest

In [ ]:
# https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
# Build a forest and compute the impurity-based feature importances
forest = ExtraTreesClassifier(n_estimators=3,
                              random_state=0)

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0) # desvio padrao da importancia de feature para cada arvore no nosso ensemble

In [ ]:
indices = np.argsort(importances)[::-1]

In [ ]:
# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X_train.columns[indices[f]], importances[indices[f]]))

In [ ]:
# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()